In [1]:
from selenium import webdriver
opts = webdriver.ChromeOptions()
opts.headless =True
from datetime import date
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import numpy as np
import time

### Checking where's the Chrome webdriver

In [2]:
try:
    browser = webdriver.Chrome(executable_path=r'C:\Users\pedro\Dropbox\pythonApplications\DataEngineering_project\chromedriver.exe',options =opts)
except:
    print('Chrome webdriver not found\n')

### Accessing the page and finding all categories

In [3]:
browser.get(r'http://books.toscrape.com/')
categories = browser.find_elements_by_class_name(r"side_categories")
if len(categories) == 0:
    print('side_categories not found\n')
    exit()
else:
    categories = np.array(categories[0].text.split('\n')[1:]).flatten()

### Navigate through categories

In [4]:
%%time
lenCategories = len(categories)

title,category,stars,price,is_in_stock = [],[],[],[],[]

for cat,i in zip(categories,range(0,lenCategories)):
    
    catPage = '-'.join(cat.split(' ')).lower()
    catPage = 'http://books.toscrape.com/catalogue/category/books/'+catPage+'_'+str(i+2)+'/index.html'
    browser.get(catPage)
    time.sleep(0.2)
    
    if browser.find_elements_by_class_name('pager') == []:
        
        # only 1 page
        response = get(browser.current_url)
        html_soup = BeautifulSoup(response.text, 'html.parser')
        book_containers = html_soup.find_all('li', class_ = 'col-xs-6 col-sm-4 col-md-3 col-lg-3')
        
        for j in range(0,len(book_containers)):
            price.append(book_containers[j].find_all('div')[1].find_all('p')[0].text.split('£')[1])
            is_in_stock.append( 'Yes' if 'In stock' in book_containers[j].find_all('div')[1].find_all('p')[1].text else 'No')
            stars.append(book_containers[j].find_all('p')[0]['class'][1])
            title.append(book_containers[j].find_all('div')[0].a.img['alt'])
            category.append(cat)
            
    else:
        
        #more than one page
        book_containers = html_soup.find_all('li', class_ = 'col-xs-6 col-sm-4 col-md-3 col-lg-3')
        
        is_last_page = False
        while is_last_page == False:
            
            response = get(browser.current_url)
            html_soup = BeautifulSoup(response.text, 'html.parser')
            book_containers = html_soup.find_all('li', class_ = 'col-xs-6 col-sm-4 col-md-3 col-lg-3')
            
            for j in range(0,len(book_containers)):
                price.append(book_containers[j].find_all('div')[1].find_all('p')[0].text.split('£')[1])
                is_in_stock.append('Yes' if 'In stock' in book_containers[j].find_all('div')[1].find_all('p')[1].text else 'No')
                stars.append(book_containers[j].find_all('p')[0]['class'][1])
                title.append(book_containers[j].find_all('div')[0].a.img['alt'])
                category.append(cat)
                
            pager = browser.find_elements_by_class_name('pager')[0]
            if pager.text.split('\n')[-1].split(' ')[0] == 'Page':
                is_last_page = True
            else:
                next_button = pager.find_elements_by_tag_name('a')
                next_button[-1].click()
                time.sleep(0.2)

Wall time: 2min 37s


In [5]:
df = pd.DataFrame({'title':title,'category':category,'stars':stars,'price':price,'is_in_stock':is_in_stock})

In [6]:
df['stars'] = df['stars'].map({'One':1,'Two':2,'Three':3,'Four':4,'Five':5})
df['title'] = df['title'].apply(lambda x: x.replace('\x80\x99',"'") if '\x80\x99' in x else x)
df['title'] = df['title'].apply(lambda x: x.replace('"',"'"))

### Checking if formatting is OK

In [7]:
## Checking if it's full of NaN

if df.isnull().sum().sum() == df.shape[0]*df.shape[1]:
    print('Only NaN values scraped. Something went wrong...\n')
    exit()

## Checking stars
nan_in_stars = df.stars.isnull().sum()
if nan_in_stars == len(df):
    print('All stars scraped are NULL. Ending execution...\n')
    exit()
    
## Checking if something differs from 'Yes', 'No' or 'Nan' in is_in_stock column
for count in df['is_in_stock'].dropna().value_counts().index:
    if df['is_in_stock'].isnull().sum() == len(df):
        print('All stars scraped are NULL. Ending execution...\n')
        exit()
    elif count !='Yes' and count !='No':
        print('Invalid value in is_in_stock column. Exiting...')
        exit()
        
## Checking if there's 'Nan' in title column
df['title'].fillna('NULL', inplace=True)

### Creating a query to insert data into scraped table

In [10]:
with open('./sql_files/insertInto_scrapedTable.sql','w', encoding="utf-8") as outFile:
    outFile.write('USE book_club_web_data;\n\n')
    outFile.write('INSERT INTO scraped(title,category,stars,price,is_in_stock) VALUES\n')
    for row in df.iloc:
        if row.name == df.iloc[-1].name:
            value = '("'+'","'.join(row.apply(lambda x: str(x)).values)+'");\n'
            value = value.replace('"NULL"','NULL')
            outFile.write(value)
        else:
            value = '("'+'","'.join(row.apply(lambda x: str(x)).values)+'"),\n'
            value = value.replace('"NULL"','NULL')
            outFile.write(value)

In [8]:
df

,title,category,stars,price,is_in_stock
0,It's Only the Himalayas,Travel,2,45.17,Yes
1,Full Moon over Noahâ's Ark: An Odyssey to Moun...,Travel,4,49.43,Yes
2,See America: A Celebration of Our National Par...,Travel,3,48.87,Yes
3,Vagabonding: An Uncommon Guide to the Art of L...,Travel,2,36.94,Yes
4,Under the Tuscan Sun,Travel,3,37.33,Yes
...,...,...,...,...,...
995,Why the Right Went Wrong: Conservatism--From G...,Politics,4,52.65,Yes
996,Equal Is Unfair: America's Misguided Fight Aga...,Politics,1,56.86,Yes
997,Amid the Chaos,Cultural,1,36.58,Yes
998,Dark Notes,Erotica,5,19.19,Yes


In [9]:
a= [1,3,5]
b= [2,4,6]

c = [(i,j) for i,j in zip(a,b)]

(1, 2)